Assignment วิเคราะห์รีวิว 1000 รีวิว

In [1]:
!pip install -U langchain-ollama langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 22.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.0 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls /content/drive/MyDrive/review_dataset

w_review_train.csv  w_review_train.parquet


In [4]:
import duckdb

con = duckdb.connect(database=':memory:', read_only=False)

In [5]:
df = con.execute("SELECT * FROM read_csv_auto('/content/drive/MyDrive/review_dataset/w_review_train.csv', parallel=false, encoding='UTF-8', ignore_errors=true)").fetchdf()

df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [6]:
df.shape

(40000, 2)

In [7]:
con.execute("COPY df TO '/content/drive/MyDrive/review_dataset/w_review_train.parquet' (FORMAT PARQUET)")

In [8]:
parquet_avg_rating_direct = con.execute("SELECT AVG(column1) FROM '/content/drive/MyDrive/review_dataset/w_review_train.parquet' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [9]:
parquet_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.parquet'

In [10]:
parquet_avg_rating_direct = con.execute(f"SELECT AVG(column1) FROM '{parquet_file_path}' WHERE column0 LIKE '%อร่อย%'").fetchone()[0]
parquet_avg_rating_direct

3.8313027179006562

In [11]:
coffee_keywords = [
    'กาแฟ', 'คาเฟ่', 'coffee', 'cafe', 'เอสเปรสโซ่', 'ลาเต้',
    'มอคค่า', 'คาปูชิโน่', 'เฟรนช์เพรส', 'cold brew', 'drip',
    'espresso', 'latte', 'americano', 'macchiato', 'brew',
    'specialty', 'slow bar', 'ร้านกาแฟ', 'ร้านคาเฟ่', 'กาแฟสด',
    'เมนูกาแฟ', 'กาแฟนม', 'กาแฟดำ', 'ร้านบาริสต้า', 'barista', 'coffee shop'
]

where_clause = " OR ".join([f"column0 LIKE '%{keyword}%'" for keyword in coffee_keywords])

In [12]:
%%time
coffee_reviews_df = con.execute(f"SELECT * FROM '{parquet_file_path}' WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.3 s, sys: 15 ms, total: 2.32 s
Wall time: 3.67 s


In [13]:
csv_file_path = '/content/drive/MyDrive/review_dataset/w_review_train.csv'

In [14]:
%%time
coffee_reviews_df_csv = con.execute(f"SELECT * FROM read_csv_auto('{csv_file_path}', parallel=false, encoding='UTF-8', ignore_errors=true) WHERE {where_clause}").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 2.58 s, sys: 60.9 ms, total: 2.64 s
Wall time: 3.79 s


In [15]:
coffee_reviews_df.head()

,column0,column1
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...,3
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...,4
2,สารภาพว่าไม่เคยคิดจะไปต่อคิวซื้อมากินเองครับ บ...,3
3,ร้านอาหารญี่ปุ่นร้านนี้ ใจจริงไม่อยากแนะนำเลยค...,5
4,เดือนแรกที่เค้าต่อคิวกัน 2 - 3 ชั่วโมง เราก็ว่...,5


In [16]:
coffee_reviews_df.shape

(4981, 2)

In [17]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [18]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [19]:
!ollama pull gemma3:270m

In [20]:
!ollama list

NAME           ID              SIZE      MODIFIED               
gemma3:270m    e7d36fb2c3b3    291 MB    Less than a second ago    


In [21]:
!ollama run gemma3:270m "สวัสดี"

สวัสดีค่ะ 😊



In [22]:
!curl -s http://127.0.0.1:11434/api/tags

{"models":[{"name":"gemma3:270m","model":"gemma3:270m","modified_at":"2025-10-19T13:54:29.409940831Z","size":291554930,"digest":"e7d36fb2c3b3293cfe56d55889867a064b3a2b22e98335f2e6e8a387e081d6be","details":{"parent_model":"","format":"gguf","family":"gemma3","families":["gemma3"],"parameter_size":"268.10M","quantization_level":"Q8_0"}}]}

In [23]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [24]:
llm = OllamaLLM(model="gemma3:270m")

In [25]:
response = llm.invoke("สวัสดี")
print(response)

สวัสดีค่ะ 😊


In [26]:
template = """
คุณเป็นครูสอน Python
คำถาม : {question}
"""

In [27]:
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

In [28]:
formatted_prompt = prompt.format(question="เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10")
print(formatted_prompt)


คุณเป็นครูสอน Python
คำถาม : เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10



In [29]:
chain = prompt | llm

In [30]:
response = chain.invoke({"question": "เขียนโค้ดหาผลรวมของตัวเลข 1 ถึง 10"})

In [31]:
response

'```python\ndef find_sum(numbers):\n    """\n    หาผลรวมของตัวเลข 1 ถึง 10\n    """\n    total = 0\n    for number in numbers:\n        total += number\n    return total\n\n# ตัวอย่างการใช้งาน\nnumbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\nresult = find_sum(numbers)\nprint(result)\n```\n\n**คำอธิบายโค้ด:**\n\n1.  **`def find_sum(numbers):`**:  กำหนดชื่อ funzione `find_sum` ที่จะใช้ในการคำนวณผลรวมของตัวเลข.\n2.  **`total = 0`**:  สร้างค่า `total` เป็นค่าคงที่ (0) และตัวแปร `numbers` เป็น iterable (list) ของตัวเลข (1, 2, 3, 4, 5, 6, 7, 8, 9, 10).\n3.  **`for number in numbers:`**:  ใช้ `for` loop เพื่อวนลูปผ่านทุกตัวเลขใน iterable `numbers`  (ซึ่งเป็น list ของตัวเลข).\n4.  **`total += number`**:  เมื่อมีการวนลูปครั้งแรกแล้ว,  `total` จะถูกเพิ่มค่าเดิม `number`  ในตัวแปร `total`  เพื่อทำให้ค่า `total`  เป็นค่าคงที่ (0).\n5.  **`return total`**:  เมื่อ `total`  ถูกเพิ่มค่าคงที่แล้ว  (ค่าคงที่)  แล้ว  `total`  จะถูกคืนค่าเป็นค่าคงที่  (0)  และ  `return total`  จะทำการคำนวณผลรวมของตัวเลขทั้งหมด

In [32]:
from IPython.display import display, Markdown

In [33]:
display(Markdown(response))

```python
def find_sum(numbers):
    """
    หาผลรวมของตัวเลข 1 ถึง 10
    """
    total = 0
    for number in numbers:
        total += number
    return total

# ตัวอย่างการใช้งาน
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum(numbers)
print(result)
```

**คำอธิบายโค้ด:**

1.  **`def find_sum(numbers):`**:  กำหนดชื่อ funzione `find_sum` ที่จะใช้ในการคำนวณผลรวมของตัวเลข.
2.  **`total = 0`**:  สร้างค่า `total` เป็นค่าคงที่ (0) และตัวแปร `numbers` เป็น iterable (list) ของตัวเลข (1, 2, 3, 4, 5, 6, 7, 8, 9, 10).
3.  **`for number in numbers:`**:  ใช้ `for` loop เพื่อวนลูปผ่านทุกตัวเลขใน iterable `numbers`  (ซึ่งเป็น list ของตัวเลข).
4.  **`total += number`**:  เมื่อมีการวนลูปครั้งแรกแล้ว,  `total` จะถูกเพิ่มค่าเดิม `number`  ในตัวแปร `total`  เพื่อทำให้ค่า `total`  เป็นค่าคงที่ (0).
5.  **`return total`**:  เมื่อ `total`  ถูกเพิ่มค่าคงที่แล้ว  (ค่าคงที่)  แล้ว  `total`  จะถูกคืนค่าเป็นค่าคงที่  (0)  และ  `return total`  จะทำการคำนวณผลรวมของตัวเลขทั้งหมด

**ตัวอย่างการใช้งาน:**

```python
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
result = find_sum(numbers)
print(result)
```

**Output:**

```
15
```

**คำอธิบาย:**

*   โค้ดนี้ใช้ `for` loop เพื่อวนลูปผ่านแต่ละตัวเลขใน iterable `numbers`  (ซึ่งเป็น list ของตัวเลข).
*   `total` จะถูกเพิ่มค่าเดิม `number`  ในตัวแปร `total`  เพื่อทำให้ค่า `total`  เป็นค่าคงที่ (0).
*   เมื่อ `total`  ถูกเพิ่มค่าคงที่แล้ว  (ค่าคงที่)  แล้ว  `total`  จะถูกคืนค่าเป็นค่าคงที่ (0)  และ  `return total`  จะทำการคำนวณผลรวมของตัวเลขทั้งหมด

**ข้อควรระวัง:**

*   การใช้ `for` loop ในการวนลูปทำให้โค้ดอาจซับซ้อนขึ้นเล็กน้อยถ้ามีตัวเลขที่ซับซ้อนใน iterable มาก
*   การใช้ `total` เป็นค่าคงที่จะทำให้การคำนวณผลรวมของตัวเลขเป็นไปอย่างถูกต้อง
*   การเพิ่มค่าคงที่ในตัวแปร `total`  จะทำให้การคำนวณผลรวมของตัวเลขเป็นไปอย่างถูกต้อง



In [34]:
prompt1 = PromptTemplate(
    template="แปลโจทย์เกี่ยวกับ {question}"
)

chain1 = prompt1 | llm

In [35]:
prompt2 = PromptTemplate(
    template="ทำความเข้าใจโจทย์ที่แปลแล้วต่อไปนี้ {translate}"
)

chain2 = prompt2 | llm

In [36]:
prompt3 = PromptTemplate(
    template="แก้โจทย์ตามที่เข้าใจต่อไปนี้ {understand}"
)

chain3 = prompt3 | llm

In [37]:
prompt4 = PromptTemplate(
    template="อธิบายวิธิีทำจากผลลัทธ์ต่อไปนี้ {solve}"
)

chain4 = prompt4 | llm

In [38]:
parser_chain = StrOutputParser()

In [39]:
full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [40]:
result = full_chain.invoke({"question":"2x + 5 = 15"})

In [41]:
result

'แน่นอนค่ะ! ผลลัทธ์เป็นเครื่องมือสำคัญในการดำเนินงานและบริหารจัดการในธุรกิจต่างๆ โดยมีบทบาทหลักๆ ดังนี้ค่ะ\n\n**ผลลัทธ์เป็นเครื่องมือที่ช่วยให้เรา:**\n\n* **กำหนดเป้าหมายและวัตถุประสงค์:** ช่วยให้เราเข้าใจความต้องการของลูกค้าและธุรกิจได้อย่างชัดเจน\n* **วางแผนและจัดการทรัพยากร:** ช่วยให้เราสามารถใช้ทรัพยากรได้อย่างมีประสิทธิภาพ\n* **ควบคุมกระบวนการ:** ช่วยให้เราสามารถควบคุมและจัดการกระบวนการต่างๆ ในธุรกิจได้อย่างมีประสิทธิภาพ\n* **ปรับปรุงประสิทธิภาพ:** ช่วยให้เราสามารถปรับปรุงประสิทธิภาพของธุรกิจได้อย่างต่อเนื่อง\n* **สร้างความสัมพันธ์กับลูกค้า:** ช่วยให้เราสร้างความสัมพันธ์ที่ดีกับลูกค้าและผู้บริโภค\n* **พัฒนาผลิตภัณฑ์และบริการ:** ช่วยให้เราพัฒนาผลิตภัณฑ์และบริการให้ตอบสนองความต้องการของลูกค้า\n\n**ประโยชน์ของผลลัทธ์:**\n\n* **การวางแผนและจัดการ:** ช่วยให้เราวางแผนและจัดการงานต่างๆ ได้อย่างมีประสิทธิภาพ\n* **การควบคุมและบริหารจัดการ:** ช่วยให้เราควบคุมและบริหารจัดการธุรกิจได้อย่างมีประสิทธิภาพ\n* **การพัฒนาผลิตภัณฑ์และบริการ:** ช่วยให้เราพัฒนาผลิตภัณฑ์และบริการให้ตอบสนองความต้องการขอ

In [42]:
import os
from langchain_openai import ChatOpenAI

In [43]:
OPENAI_API_KEY = "xxx"

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [44]:
llm2 = ChatOpenAI(model="gpt-4o-mini")

In [45]:
chain1 = prompt1 | llm2
chain2 = prompt2 | llm2
chain3 = prompt3 | llm2
chain4 = prompt4 | llm2

full_chain = chain1 | chain2 | chain3 | chain4 | parser_chain

In [46]:
formatted_result = result.replace('\\[', '$$') \
                         .replace('\\]', '$$') \
                         .replace('\\(', '$') \
                         .replace('\\)', '$')

In [47]:
display(Markdown(formatted_result))

แน่นอนค่ะ! ผลลัทธ์เป็นเครื่องมือสำคัญในการดำเนินงานและบริหารจัดการในธุรกิจต่างๆ โดยมีบทบาทหลักๆ ดังนี้ค่ะ

**ผลลัทธ์เป็นเครื่องมือที่ช่วยให้เรา:**

* **กำหนดเป้าหมายและวัตถุประสงค์:** ช่วยให้เราเข้าใจความต้องการของลูกค้าและธุรกิจได้อย่างชัดเจน
* **วางแผนและจัดการทรัพยากร:** ช่วยให้เราสามารถใช้ทรัพยากรได้อย่างมีประสิทธิภาพ
* **ควบคุมกระบวนการ:** ช่วยให้เราสามารถควบคุมและจัดการกระบวนการต่างๆ ในธุรกิจได้อย่างมีประสิทธิภาพ
* **ปรับปรุงประสิทธิภาพ:** ช่วยให้เราสามารถปรับปรุงประสิทธิภาพของธุรกิจได้อย่างต่อเนื่อง
* **สร้างความสัมพันธ์กับลูกค้า:** ช่วยให้เราสร้างความสัมพันธ์ที่ดีกับลูกค้าและผู้บริโภค
* **พัฒนาผลิตภัณฑ์และบริการ:** ช่วยให้เราพัฒนาผลิตภัณฑ์และบริการให้ตอบสนองความต้องการของลูกค้า

**ประโยชน์ของผลลัทธ์:**

* **การวางแผนและจัดการ:** ช่วยให้เราวางแผนและจัดการงานต่างๆ ได้อย่างมีประสิทธิภาพ
* **การควบคุมและบริหารจัดการ:** ช่วยให้เราควบคุมและบริหารจัดการธุรกิจได้อย่างมีประสิทธิภาพ
* **การพัฒนาผลิตภัณฑ์และบริการ:** ช่วยให้เราพัฒนาผลิตภัณฑ์และบริการให้ตอบสนองความต้องการของลูกค้า
* **การสร้างความสัมพันธ์กับลูกค้า:** ช่วยให้เราสร้างความสัมพันธ์ที่ดีกับลูกค้าและผู้บริโภค
* **การปรับปรุงประสิทธิภาพ:** ช่วยให้เราปรับปรุงประสิทธิภาพของธุรกิจได้อย่างต่อเนื่อง

**ตัวอย่างการใช้ผลลัทธ์:**

* **การจัดทำแผนธุรกิจ:** กำหนดเป้าหมายและวัตถุประสงค์ของธุรกิจ
* **การวางแผนการตลาด:** กำหนดกลยุทธ์การตลาดและช่องทางการตลาด
* **การบริหารจัดการงบประมาณ:** กำหนดงบประมาณและติดตามผลการดำเนินงาน
* **การจัดการการเงิน:** จัดทำและติดตามการจัดการเงินทุน
* **การบริหารความเสี่ยง:** วิเคราะห์และจัดการความเสี่ยงด้านต่างๆ

**สรุป:**

ผลลัทธ์เป็นเครื่องมือที่สำคัญอย่างยิ่งในการดำเนินงานและบริหารจัดการธุรกิจต่างๆ โดยช่วยให้เราเข้าใจความต้องการของลูกค้าและธุรกิจได้อย่างชัดเจน ลดความเสี่ยง และพัฒนาผลิตภัณฑ์และบริการให้ตอบสนองความต้องการของลูกค้า

หวังว่าคำอธิบายนี้จะช่วยให้คุณเข้าใจวิธิีทำจากผลลัทธ์ได้ดีขึ้นนะคะ! 😊

In [48]:
prompt_template = """
[1. Role/Context บอก LLM ว่าควรทำตัวเป็นอะไร]
คุณเป็น AI ที่เชี่ยวชาญในการวิเคราะห์รีวิวภาษาไทย

[2. Task Description บอกงานที่ต้องทำ]
จาก Review ต่อไปนี้ ให้ Extract 3 keywords
ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

[3. Input Data ข้อมูลที่ต้องประมวลผล]
Review: {review}

[4. Output Format บอกรูปแบบของคำตอบ]
ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}

[5. Constraints ข้อจำกัดหรือเงื่อนไข]
- ต้องเป็นคำภาษาไทย
- คั่นด้วย comma
- หากไม่พบให้ตอบ 'ไม่ระบุ'

[6. Examples (optional) ตัวอย่าง input/output]
ตัวอย่าง
Input: "กาแฟอร่อย บรรยากาศดี ราคาแพง"
Output: {"keywords": "กาแฟ, บรรยากาศ, ราคา"}
"""

In [49]:
prompt = """
Extract keywords from this review
{review}
"""

# Pros Simple, fast
# Cons อาจได้ผลไม่ดีกับ Tasks ซับซ้อน

In [50]:
prompt = """
Extract keywords from reviews.

Example 1
Review: "กาแฟอร่อย บริการดี"
Keywords: กาแฟ, บริการ

Example 2
Review: "ราคาแพง แต่คุ้มค่า"
Keywords: ราคา, คุณภาพ

Now extract from this review
Review: {review}
Keywords:
"""

# Pros ได้ผลดีกว่า Zero-shot
# Cons Prompt ใช้ Tokens มากขึ้น

In [51]:
prompt = """
Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON: {{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
"""

# Pros ได้ผลดีกับ Reasoning tasks
# Cons ใช้ Tokens มากกว่า ช้ากว่า

In [52]:
keyword_prompt = PromptTemplate(
template = """
จาก Review ต่อไปนี้
Extract 3 keywords ที่เป็นตัวแทนของ Review นี้

Review: {review}

ขั้นตอนการ Extract keywords
1. อ่านรีวิวและเข้าใจความหมาย
2. ระบุหัวข้อหลักที่กล่าวถึง
3. เลือก 3 keywords ที่สำคัญที่สุด
4. ตอบในรูปแบบ JSON โดยมี key คือ keyword เช่น :{{"keywords": "คำ1, คำ2, คำ3"}}

ให้ทำทีละขั้นตอน
""")

In [53]:
sample_reviews = coffee_reviews_df['column0'].head(2)
sample_reviews

,column0
0,ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้...
1,วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอา...


In [54]:
sample_reviews.to_csv('/content/sample_reviews.csv', index=False)

In [55]:
keyword_extract = keyword_prompt | llm | StrOutputParser()

In [56]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 36.3 ms, sys: 3.26 ms, total: 39.6 ms
Wall time: 5.78 s


In [57]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
"keywords": ["ร้านอาหาร", "ห้องน้ำ", "กาแฟ", "น้ำผึ้ง", "น้ำผึ้ง", "ลาบไข่ต้ม", "ไข่มัน", "ขนมหวาน", "ขนมไทย", "ขนม"]
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "ร้อน",
    "แก้วนึง",
    "ร้อน",
    "วางอยู่ข้างหน้า",
    "เล่นลายฟองนม",
    "เว้ยยย",
    "ซากอารยธรรมลาเต้",
    "เมพพพพ",
    "มารสชาติและกลิ่น",
    "มวยรุ่นเดียวกะ Starbuck"
  ]
}
```



In [58]:
keyword_extract = keyword_prompt | llm2 | StrOutputParser()

In [59]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
"keywords": ["ร้านอาหาร", "ห้องน้ำ", "กาแฟ", "น้ำผึ้ง", "น้ำผึ้ง", "ลาบไข่ต้ม", "ไข่มัน", "ขนมหวาน", "ขนมไทย", "ขนม"]
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": [
    "ลาเต้",
    "ร้อน",
    "แก้วนึง",
    "ร้อน",
    "วางอยู่ข้างหน้า",
    "เล่นลายฟองนม",
    "เว้ยยย",
    "ซากอารยธรรมลาเต้",
    "เมพพพพ",
    "มารสชาติและกลิ่น",
    "มวยรุ่นเดียวกะ Starbuck"
  ]
}
```



In [60]:
!ollama pull scb10x/typhoon2.1-gemma3-4b

In [61]:
!ollama list

NAME                                  ID              SIZE      MODIFIED               
scb10x/typhoon2.1-gemma3-4b:latest    8cfab9097c9d    2.6 GB    Less than a second ago    
gemma3:270m                           e7d36fb2c3b3    291 MB    About a minute ago        


In [62]:
llm3 = OllamaLLM(model="gemma3:270m")

In [63]:
keyword_extract = keyword_prompt | llm3 | StrOutputParser()

In [64]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    cleaned = (text).replace("**", "").strip()
    results.append({"review": review, "text": cleaned})

CPU times: user 62.9 ms, sys: 3.58 ms, total: 66.4 ms
Wall time: 5.28 s


In [65]:
for result in results:
    print(f"Review {result['review'][:100]}...")
    print(f"text {result['text']}\n")

Review ร้านอาหารใหญ่มากกกกกกก 
เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี 
ห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ...
text ```json
{
  "keywords": ["ร้านอาหาร", "ห้องน้ำ", "กาแฟ", "น้ำผึ้ง", "น้ำผึ้ง", "ลาบไข่ต้ม", "ไข่มัน", "ขนมหวาน", "ขนมไทย", "ขนมผลไม้", "ขนมปลอมจิน", "ขนมปรก", "ขนมหวานฟรี", "ขนมไทย", "ขนมผลไม้", "ขนมปลอมจิน", "ขนมปรก", "ขนมปรกฟรี", "ขนมไทย", "ขนม"]
}
```

Review วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์

ก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก ...
text ```json
{
  "keywords": ["ลาเต้", "ร้อน", "แก้วนึง", "เสริฟ", "หุๆ", "เมพพพ", "ซากอารยธรรม", "เมพพพ", "กาแฟ", "ราคาไม่แพง", "กินบ่อย", "จุกได้", "ร้านวาวี", "อารีย์", "การชิม", "ถ่ายโดย", "ซัมซวย", "Omnia i900", "สีภาพถึงได้ไม่ค่อยสวยเท่าไหร", "ขั้นตอนการ Extract keywords", "รีวิวและเข้าใจความหมาย", "ระบุหัวข้อหลักที่กล่าวถึง", "เลือก 3 keywords ที่สำคัญที่สุด", "ตอบในรูปแบบ JSON โดยมี key คือ keyword", "ให้ทำทีละขั้นตอน"
}
```



In [66]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class ReviewKeywords(BaseModel):
    keywords: str = Field(
        ...,  # Required field
        description="3 keywords separated by comma",
        examples=["กาแฟ, บรรยากาศ, ราคา"]
    )

class KeywordsExtraction(BaseModel):
    extracted_qualities: ReviewKeywords

In [67]:
keywords_parser = JsonOutputParser(pydantic_object=KeywordsExtraction)

In [68]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [69]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 56.5 ms, sys: 3.84 ms, total: 60.3 ms
Wall time: 6.4 s


In [70]:
results[0]['text']['keywords']

['ร้านอาหารใหญ่',
 'ห้องน้ำ',
 'กาแฟ',
 'น้ำผึ้ง',
 'น้ำผึ้งมาราด',
 'แพงเวอร์',
 'อย่าสั่งเลย',
 'ลาบไข่ต้ม',
 'ไข่มันคาว',
 'ไม่ประทับใจ',
 'ราคา150บาท']

In [71]:
results[1]['text']['keywords']

['ลาเต้',
 'ร้อน',
 'ร้อน',
 'กาแฟ',
 'ร้านวาวี',
 'Starbuck',
 'แฟนๆ Starbuck',
 'สวย',
 'อร่อย',
 'ราคาไม่แพง',
 'กินบ่อย',
 'จุกได้',
 'ซากอารยธรรมลาเต้',
 'เมพพพ',
 'เมพพพพ',
 'ซีเรียส',
 'กาแฟ']

In [72]:
llm3 = OllamaLLM(
    model="scb10x/typhoon2.1-gemma3-4b",
    temperature=0.05
)

keyword_extract = keyword_prompt | llm3 | keywords_parser

In [73]:
%%time
results = []
for review in sample_reviews:
    text = keyword_extract.invoke({"review": review})
    results.append({"review": review, "text": text})

CPU times: user 178 ms, sys: 16.7 ms, total: 195 ms
Wall time: 18.4 s


In [74]:
results[0]['text']['keywords'], results[1]['text']['keywords']

(['ราคา', 'รสชาติ', 'ขนาด'], ['กาแฟ', 'บรรยากาศ', 'วาวี'])

In [75]:
keyword_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract keywords 3 keywords เท่านั้น ที่เป็นตัวแทนของ Review นี้ได้ดีที่สุด

    Review: {review}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'keywords'
    และค่าเป็นข้อความที่ดึงได้คั่นด้วย comma
    เช่น {{"keywords": "กาแฟ, บรรยากาศ, ราคา"}}

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [76]:
keyword_extract = keyword_prompt | llm3 | keywords_parser

In [77]:
inputs = [{"review": r} for r in sample_reviews]

In [78]:
%%time
results = keyword_extract.batch(inputs)

CPU times: user 27 ms, sys: 1.08 ms, total: 28.1 ms
Wall time: 3.09 s


In [79]:
results

[{'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
 {'keywords': 'กาแฟ, บรรยากาศ, รสชาติ'}]

In [80]:
sample_reviews_1000 = coffee_reviews_df['column0'].head(100).tolist()
inputs_1000 = [{"review": r} for r in sample_reviews_1000]

BATCH_SIZE = 8

results_1000 = []

In [81]:
from tqdm import tqdm

In [82]:
for i in tqdm(range(0, len(inputs_1000), BATCH_SIZE), desc="Extracting keywords"):
    batch = inputs_1000[i:i+BATCH_SIZE]
    outputs = keyword_extract.batch(batch)
    for inp, out in zip(batch, outputs):
        results_1000.append({"review": inp["review"], "text": out})

Extracting keywords: 100%|██████████| 13/13 [02:19<00:00, 10.76s/it]


In [83]:
for result in results_1000[0:10]:
    print(result['text']['keywords'])

ราคา, บรรยากาศ, ไข่ต้ม
กาแฟ, บรรยากาศ, รสชาติ
โดนัท, รสชาติ, ความหวาน
วาซาบิ, ปลาซาบะ, ข้าวหน้าปลา
โดนัท, Original Glazed, คิว
วังพญาไท, นรสิงห์, คาเฟ่
อาหารทะเล, ราคา, บรรยากาศ
กาแฟ, เค้ก, พนักงาน
ฮันนี่ โทสท์, ราคา, รสชาติ
คาเฟ่, วานิลลา, คาโบนาร่า


In [84]:
  results_1000[0]

{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
 'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'}}

In [85]:
context_prompt = PromptTemplate(
    template="""จาก Review ต่อไปนี้
    Extract context ใน Review ที่เป็นตัวแทนที่บรรยายคำว่า {keyword} ได้ดีที่สุด

    Review: {review}

    ตอบผลลัพธ์ในรูปแบบ JSON โดยมีคีย์เป็น 'context'
    และค่าเป็นข้อความที่ดึงได้จาก Review

    ตัวอย่าง
    keyword คือ "กาแฟ"
    context: "กาแฟที่นี่อร่อยมาก หอมกลิ่นกาแฟคั่วสด"

    หากไม่พบข้อมูลให้ตอบ 'ไม่ระบุ'
    """
)

In [86]:
sentiment_prompt = PromptTemplate(
    template="""จากข้อความต่อไปนี้ ให้วิเคราะห์ว่าเป็น sentiment แบบไหน
    เลือกจากตัวเลือกเหล่านี้เท่านั้น: "positive", "negative", "neutral"

    ข้อความ: {message}

    ตอบในรูปแบบ JSON โดยมีคีย์เป็น 'sentiment'
    และค่าเป็นหนึ่งใน list ต่อไปนี้ [positive, negative, neutral]

    คำแนะนำ
    - positive คือ ข้อความที่แสดงความพอใจ ชอบ ดี
    - negative คือ ข้อความที่แสดงความไม่พอใจ ไม่ชอบ แย่
    - neutral คือ ข้อความที่ไม่แสดงความรู้สึกชัดเจน เป็นกลาง

    # ตัวอย่าง
    # ข้อความ "อร่อยมาก" → {{"sentiment": "positive"}}
    # ข้อความ "แพงไป" → {{"sentiment": "negative"}}
    # ข้อความ "ปกติ" → {{"sentiment": "neutral"}}
    """
)


In [87]:
class ContextKeyword(BaseModel):
    context: str = Field(..., description="context that best represent this keyword")

class ContexExtraction(BaseModel):
    extracted: ContextKeyword

context_parser = JsonOutputParser(pydantic_object=ContexExtraction)

In [88]:
context_extract = context_prompt | llm3 | context_parser

In [89]:
class Sentiment(BaseModel):
    sentiment: str = Field(..., description="sentiment of review")

class SentimentExtraction(BaseModel):
    extracted: Sentiment


sentiment_parser = JsonOutputParser(pydantic_object=SentimentExtraction)

In [90]:
sentiment_extract = sentiment_prompt | llm3 | sentiment_parser

In [91]:
keywords = results_1000[0]['text']['keywords']


In [92]:
context_inputs = []

for review_data in tqdm(results_1000, desc="Preparing context inputs"):
    review = review_data['review']
    keywords = review_data['text']['keywords']
    for keyword in keywords:
        context_inputs.append({"review": review, "keyword": keyword.strip()})

Preparing context inputs: 100%|██████████| 100/100 [00:00<00:00, 50945.03it/s]


In [93]:
len(context_inputs)

2522

In [94]:
context_results = []
for i in tqdm(range(len(results_1000)), desc="Extracting contexts"):
   result = context_extract.batch(context_inputs[i*3:(i*3)+3])
   contexts_list = [d['context'] for d in result]
   results_1000[i]['contexts'] = contexts_list

Extracting contexts: 100%|██████████| 100/100 [09:46<00:00,  5.87s/it]


In [95]:
results_1000[:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ร้านอาหารใหญ่มากกกกกกก',
   'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี',
   'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยากาศรอบๆร้านก็แต่งได้น่านั่ง อารมณ์ประมาณว่าอยู่ในสว

In [96]:
sentiment_inputs = []

for review_data in tqdm(results_1000, desc="Preparing sentiment inputs"):
    contexts = review_data['contexts']
    for context in contexts:
        sentiment_inputs.append({"message": context})

Preparing sentiment inputs: 100%|██████████| 100/100 [00:00<00:00, 174908.42it/s]


In [97]:
len(sentiment_inputs)

300

In [98]:
sentiment_inputs[:6]

[{'message': 'ร้านอาหารใหญ่มากกกกกกก'},
 {'message': 'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี'},
 {'message': 'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย'},
 {'message': 'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี'},
 {'message': 'ราคา 150 บาทมันเกินไปนะ รับไม่ไหวว'},
 {'message': 'ทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล'}]

In [99]:
for i in tqdm(range(len(results_1000)), desc="Extracting sentiment"):
   result = sentiment_extract.batch(sentiment_inputs[i*3:(i*3)+3])
   sentiment_list = [d['sentiment'] for d in result]
   results_1000[i]['sentiment'] = sentiment_list

Extracting sentiment: 100%|██████████| 100/100 [04:57<00:00,  2.98s/it]


In [100]:
results_1000[0:2]

[{'review': 'ร้านอาหารใหญ่มากกกกกกก \nเลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี \nห้องทานหลักๆอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย \nลาบไข่ต้ม ไข่มันคาวอะ เลยไม่ประทับใจเท่าไหร่\nทอดมันหัวปลีกรอบอร่อยต้องเบิ้ล \nพะแนงห่อไข่อร่อยดี เห้ยแต่ราคา 150บาทมันเกินไปนะ รับไม่ไหวว\nเลิกกินแล้วมีขนมหวานให้กินฟรีเล็กน้อย )ขนมไทย) \n\nคงไม่ไปซ้ำ แพงเกิน ',
  'text': {'keywords': 'ราคา, บรรยากาศ, ไข่ต้ม'},
  'contexts': ['ร้านอาหารใหญ่มากกกกกกก',
   'ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้ำก่อนเลย เออแปลกดี',
   'ห้องทานอาหารอยู่ชั้น 2 มีกาแฟ น้ำผึ้ง ซึ่งก็แค่เอาน้ำผึ้งมาราด แพงเวอร์ อย่าสั่งเลย'],
  'sentiment': ['neutral', 'neutral', 'negative']},
 {'review': 'วันนี้ได้มีโอกาสไปนั่งซดกาแฟที่ร้านวาวี แถวๆอารีย์\n\nก็ได้ยินมาบ่อยๆละ จากหลายที่ แต่ที่บ่อยสุดก็จาก Twitter ว่ากาแฟที่นี่อร่อยมากกกกกกก\nเรียกว่าระดับแฟนๆ Starbuck ยังต้องเหลียวมามอง ก็ดองมานานจนถึงวันนี้โอกาสเหมาะ ไป\npresent งานที่ตึก Software Park เสร็จก็เลยมาหาไรกินแถวนี้พอดี\nจัดไป.....\n\n\nบรรยา

In [101]:
import pandas as pd

df = pd.DataFrame([
    {
        'review': r['review'],
        **{f'keyword{i+1}': k.strip() for i, k in enumerate(r['text']['keywords'])},
        **{f'context{i+1}_keyword': c for i, c in enumerate(r['contexts'])},
        **{f'sentiment{i+1}': s for i, s in enumerate(r['sentiment'])}
    }
    for r in results_1000
])

In [102]:
df.to_csv('results.csv', index=False, encoding='utf-8-sig')